In [2]:
import os

In [3]:
%pwd

'a:\\MlProject\\RED_WINE_QUALITY\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'a:\\MlProject\\RED_WINE_QUALITY'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    status_file: str
    unzip_data_dir: Path
    all_schema: dict

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path


In [7]:

from Red_Wine_Quality.constants.constant import *
from Red_Wine_Quality.utils.common import read_yaml, create_directories


In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

    def get_data_validation_config(self) -> DataValidationConfig:
        config=self.config.data_validation
        schema=self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config=DataValidationConfig(
            root_dir=config.root_dir,
            status_file=config.status_file,
            all_schema=schema,
            unzip_data_dir=config.unzip_data_dir
        )


        return data_validation_config

In [9]:
import os 
from Red_Wine_Quality.logging.logger import logger
import pandas as pd

In [18]:
class DataValidation():
    def __init__(self,config: DataValidationConfig):
        self.config=config

    def validate_all_columns(self)-> bool:
        try:
            validate_col_status=None
            data=pd.read_csv(self.config.unzip_data_dir)
            all_cols=list(data.columns)
            all_schema=self.config.all_schema.keys()
            

            for col in all_cols:
                if col not in all_schema:
                    validate_col_status=False
                    
                    with open(self.config.status_file,'a') as f:
                        f.write(f"Validation(DATA) {col} Status: {validate_col_status}\n")
                else:
                    validate_col_status=True
                    with open(self.config.status_file,'a') as f:
                        f.write(f"Validation(DATA) {col} Status: {validate_col_status} \n")
            
        
            return validate_col_status
        except Exception as e:
            raise e

    def validate_all_data_types(self) -> bool:
        try:
            validate_dtype_status=None
            data=pd.read_csv(self.config.unzip_data_dir)
            all_dtpyes=list(data.dtypes)
            all_schema=self.config.all_schema

            list_schema_key=list(self.config.all_schema.keys())
            col=0
            for dtype in (all_dtpyes):
                
                if dtype not in all_schema.values():
                    validate_dtype_status=False
                    with open(self.config.status_file,'a') as f:
                        f.write(f"Validation(DTPYE) of {list_schema_key[col]} and dtpye {dtype} Status: {validate_dtype_status}\n")
                else:
                    validate_dtype_status=True
                    with open(self.config.status_file,'a') as f:
                        f.write(f"Validation(DTYPE) of {list_schema_key[col]} and dtpye {dtype} Status: {validate_dtype_status}\n")
                col=col+1
            
            

            return validate_dtype_status
        except Exception as e:
            raise e









    

In [19]:
try:
    config=ConfigurationManager()
    data_validation_config=config.get_data_validation_config()
    data_validation=DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
    data_validation.validate_all_data_types()
except Exception as e:
    raise e    

[2024-04-24 13:45:33,564: INFO: common: yaml file: A:\MlProject\RED_WINE_QUALITY\config\config.yaml loaded successfully]
[2024-04-24 13:45:33,566: INFO: common: yaml file: A:\MlProject\RED_WINE_QUALITY\params.yaml loaded successfully]
[2024-04-24 13:45:33,569: INFO: common: yaml file: A:\MlProject\RED_WINE_QUALITY\schema.yaml loaded successfully]
[2024-04-24 13:45:33,571: INFO: common: created directory at: artifacts]
[2024-04-24 13:45:33,571: INFO: common: created directory at: artifacts/data_validation]
